In [1]:
from model import DQN
import os
import minerl
import gym
import torch
import torch.nn.functional as F
import torch.optim as optim
import ray
import pandas as pd
import gc
import asyncio
from _collections import deque
from utils import *
import random

def learner_append_sample(memory, model, target_model, state, action, reward, next_state, done):
    # Caluclating Priority (TD Error)
    target = model(state.float()).data.cpu()
    old_val = target[0][action].cpu()
    target_val = target_model(next_state.float()).data.cpu()
    if done:
        target[0][action] = reward
    else:
        target[0][action] = reward + 0.99 * torch.max(target_val)

    error = abs(old_val - target[0][action])
    error = error.cpu()
    memory.add.remote(error, [state, action, reward, next_state, done])



/home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)
/home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
@ray.remote
class ParameterServer:
    def update_parameters(self, learner_params): 
        self.learner_params = learner_params

    def return_parameters(self):
        return self.learner_params

    def return_saving_status(self):
        return self.is_saved

In [3]:
@ray.remote
class Actor:
    def __init__(self, learner, actor_idx, startEpsilon, endEpsilon):
        # environment initialization
        self.actor_idx = actor_idx
        self.env = gym.make("MineRLTreechop-v0")
        self.port_number = int("12340") + actor_idx
        print("actor environment %d initialize successfully" % self.actor_idx)
        self.env.make_interactive(port=self.port_number, realtime=False)
        self.shared_network_cpu = ray.get(learner.get_network.remote())
        # self.shared_memory = ray.get(shared_memory_id)
        # print("shared memory assign successfully")
        
        # network initalization
        self.actor_network = DQN(19).cpu()
        self.actor_target_network = DQN(19).cpu()
        self.actor_network.load_state_dict(self.shared_network_cpu.state_dict())
        self.actor_target_network.load_state_dict(self.actor_network.state_dict())
        print("actor network %d initialize successfully" % self.actor_idx)

        self.initialized = False
        self.epi_counter = 0
        # exploring info
        self.startEpsilon = startEpsilon
        self.endEpsilon = endEpsilon
        self.max_episodes = 1000



    # 1. 네트워크 파라미터 복사
    # 2. 환경 탐험 (초기화, 행동)
    # 3. 로컬버퍼에 저장
    # 4. priority 계산
    # 5. 글로벌 버퍼에 저장
    # 6. 주기적으로 네트워크 업데이트

    def get_initialized(self):
        return self.initialized

    def get_counter(self):
        return self.epi_counter

    # 각 환경 인스턴스에서 각 엡실론에 따라 탐험을 진행한다.
    # 탐험 과정에서 local buffer에 transition들을 저장한다.
    # local buffer의 개수가 특정 개수 이상이면 global buffer에 추가해준다.

    def explore(self, learner, shared_memory):
        
        self.initialized = True
        stepDrop = (self.startEpsilon - self.endEpsilon) / self.max_episodes
        epsilon = self.startEpsilon
        total_steps = 0
        
        episodes = [x for x in range(self.max_episodes)]
        train_stats = pd.DataFrame(index=episodes, columns=['rewards'])
        
        for num_epi in range(self.max_episodes):
            obs = self.env.reset()
            state = converter(obs).cpu().float()
            done = False
            total_reward = 0
            steps = 0
            if (epsilon > self.endEpsilon):
                epsilon -= stepDrop
                
            while not done:
                steps += 1
                total_steps += 1
                a_out = self.actor_network.sample_action(state, epsilon)
                action_index = a_out
                action = make_action(self.env, action_index)
                obs_prime, reward, done, info = self.env.step(action)
                total_reward += reward
                state_prime = converter(obs_prime)

                self.actor_append_sample(shared_memory, self.actor_network, self.actor_target_network, \
                                       state, action_index, reward, state_prime, done)

                state = state_prime.float().cpu()
                if done:
                    break
                    
            print(ray.get(shared_memory.size.remote()))

            # pandas로 리워드 기록하기
            print("%d episode is done" % num_epi)
            print("total rewards : %d " % total_reward)
            train_stats.loc[num_epi]['rewards'] = total_reward
            train_stats.to_csv('train_stat_minerl_agent {}.csv'.format(str(self.actor_idx)))


            #if (num_epi % 2 == 0 and num_epi != 0):
            shared_network = ray.get(learner.get_network.remote())
            self.actor_network.load_state_dict(shared_network.state_dict())
            print("actor network is updated ")

    def env_close(self):
        self.env.close()        

    def actor_append_sample(self, memory, model, target_model, state, action, reward, next_state, done):
        # Caluclating Priority (TD Error)
        target = model(state.float()).data.cpu()
        old_val = target[0][action].cpu()
        target_val = target_model(next_state.float()).data.cpu()
        if done:
            target[0][action] = reward
        else:
            target[0][action] = reward + 0.99 * torch.max(target_val)

        error = abs(old_val - target[0][action])
        error = error.cpu()
        memory.add.remote(error, [state, action, reward, next_state, done])






In [4]:
@ray.remote(num_gpus=1)
class Learner:
    def __init__(self, network, batch_size):
        self.learner_network = DQN(19).cuda().float()
        self.learner_target_network = DQN(19).cuda().float()
        self.learner_network.load_state_dict(network.state_dict())
        self.learner_target_network.load_state_dict(network.state_dict())
        self.shared_network = DQN(19).cpu()
        self.shared_target_network = DQN(19).cpu()
        
        self.count = 0
        self.batch_size = batch_size
        self.max_counts= 1000000

    # 1. sampling
    # 2. calculate gradient
    # 3. weight update
    # 4. compute priorities
    # 5. priorities of buffer update
    # 6. remove old memory
    def count(self):
        return self.count
    
    def get_network(self):
        self.shared_network.load_state_dict(self.learner_network.state_dict())
        print("return learner network")
        return self.shared_network
    
    def get_target_network(self):
        self.shared_target_network.load_state_dict(self.learner_target_network.state_dict())
        return self.shared_target_network

    def update_network(self, memory, demos, batch_size, optimizer):
        print("started")
        print(ray.get(memory.size.remote()))
        print(ray.get(demos.size.remote()))
        
        counts = [x for x in range(self.max_counts)]
        train_stats = pd.DataFrame(index=counts, columns=['loss'])
        while(self.count < 1000000):
           

            agent_batch, agent_idxs, agent_weights = ray.get(memory.sample.remote(batch_size))
            demo_batch, demo_idxs, demo_weights = ray.get(demos.sample.remote(batch_size))

            # demo_batch = (batch_size, state, action, reward, next_state, done, n_rewards)
            # print(len(demo_batch[0])) # 0번째 배치이므로 0이 나옴
            state_list = []
            action_list = []
            reward_list = []
            next_state_list = []
            done_mask_list = []

            print("agent batch len : {} ".format(str(len(agent_batch))))
            for agent_transition in agent_batch:
                s, a, r, s_prime, done_mask = agent_transition
                state_list.append(s)
                action_list.append([a])
                reward_list.append([r])
                next_state_list.append(s_prime)
                done_mask_list.append([done_mask])

            print("demo batch len : {} ".format(str(len(demo_batch))))
            
            for expert_transition in demo_batch:
                s, a, r, s_prime, done_mask = expert_transition
                state_list.append(s)
                action_list.append([a])
                reward_list.append([r])
                next_state_list.append(s_prime)
                done_mask_list.append([done_mask])

            s = torch.stack(state_list).float().cuda()
            a = torch.tensor(action_list, dtype=torch.int64).cuda()
            r = torch.tensor(reward_list).cuda()
            s_prime = torch.stack(next_state_list).float().cuda()
            done_mask = torch.tensor(done_mask_list).float().cuda()

            q_vals = self.learner_network(s)
            state_action_values = q_vals.gather(1, a)

            # comparing the q values to the values expected using the next states and reward
            next_state_values = self.learner_target_network(s_prime).max(1)[0].unsqueeze(1)
            target = r + (next_state_values * gamma * done_mask)

            # calculating the q loss, n-step return lossm supervised_loss
            is_weights = torch.FloatTensor(agent_weights).to(device)
            q_loss = (is_weights * F.mse_loss(state_action_values, target)).mean()
            #supervised_loss = margin_loss(q_vals, a, 1, 1)

            loss = q_loss #+ supervised_loss
            errors = torch.abs(state_action_values - target).data.cpu().detach()
            errors = errors.numpy()
            # update priority
            for i in range(batch_size):
                idx = agent_idxs[i]
                memory.update.remote(idx, errors[i])

            train_stats.loc[self.count ]['loss'] = float(loss.item())
            train_stats.to_csv('train_stat_minerl_learner.csv')

            # optimization step and logging
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            torch.save(self.learner_network.state_dict(), model_path + "apex_dqfd_learner.pth")
            self.count +=1
            if(self.count % 100 == 0 and self.count != 0):
                self.learner_target_network.load_state_dict(self.learner_network.state_dict())
                print("Count : {} leaner_target_network updated".format(self.count))
                
            if(self.count % 10 == 0 and self.count!= 0):
                print("Count : {} leaner_network updated".format(self.count))
  




In [5]:
@ray.remote
def parse_demo(env_name, rep_buffer, policy_net, target_net, threshold=10, num_epochs=1, batch_size=16,
      seq_len=10, gamma=0.99):
    data = minerl.data.make(env_name)
    print("data loading sucess")
    demo_num = 0
    parse_ts = 0

    for s_batch, a_batch, r_batch, ns_batch, d_batch in data.batch_iter(num_epochs=num_epochs, batch_size=batch_size,seq_len=seq_len):
        print(ray.get(rep_buffer.size.remote()))
        if(ray.get(rep_buffer.size.remote()) > 9999):
            del data
            return

        demo_num += 1
        print(demo_num)
        print(r_batch.sum())
        if r_batch.sum() < threshold:
            del s_batch, a_batch, r_batch, d_batch, ns_batch
            continue


        batch_length = (s_batch['pov'].shape)[0]  # (batch, seq, 64, 64, 3)[0]
        for i in range(0, batch_length):
            episode_start_ts = 0

            for j in range(0, seq_len):
                av = a_batch['attack'][i][j]  # attack value
                aj = a_batch['jump'][i][j]  # jump value
                af = a_batch['forward'][i][j]  # forward value
                ab = a_batch['back'][i][j]  # back value
                al = a_batch['left'][i][j]  # left value
                ar = a_batch['right'][i][j]  # right value
                va = a_batch['camera'][i][j][0]  # vertical angle and
                ha = a_batch['camera'][i][j][1]  # horizontal angle

                camera_thresholds = (abs(va) + abs(ha)) / 2.0
                # 카메라를 움직이는 경우
                if (camera_thresholds > 2.5):
                    # camera = [0, -5]
                    if abs(va) < abs(ha) and ha < 0:
                        if av == 0:
                            action_index = 0
                        else:
                            action_index = 1
                    # camera = [0, 5]
                    elif abs(va) < abs(ha) and ha > 0:
                        if av == 0:
                            action_index = 2
                        else:
                            action_index = 3
                    # camera = [-5, 0]
                    elif abs(va) > abs(ha) and ha < 0:
                        if av == 0:
                            action_index = 4
                        else:
                            action_index = 5
                    # camera = [5, 0]
                    elif abs(va) > abs(ha) and ha > 0:
                        if av == 0:
                            action_index = 6
                        else:
                            action_index = 7

                            # 카메라를 안움직이는 경우
                # 점프하는 경우
                elif (aj == 1):
                    if (af == 0):
                        action_index = 8
                    else:
                        action_index = 9

                # 앞으로 가는 경우
                elif (af == 1):
                    if (av == 0):
                        action_index = 10
                    else:
                        action_index = 11

                # 뒤로 가는 경우
                elif (ab == 1):
                    if (av == 0):
                        action_index = 12
                    else:
                        action_index = 13

                # 왼쪽으로 가는 경우
                elif (al == 1):
                    if (av == 0):
                        action_index = 14
                    else:
                        action_index = 15

                # 오른쪽으로 가는 경우
                elif (ar == 1):
                    if (av == 0):
                        action_index = 16
                    else:
                        action_index = 17

                # 카메라, 움직임이 다 0이고 공격만 하는 것
                else:
                    if (av == 0):
                        continue
                    else:
                        action_index = 18

                a_index = torch.LongTensor([action_index]).cpu()
                state = converter2(s_batch['pov'][i][j]).float().cpu()
                next_state = converter2(ns_batch['pov'][i][j]).float().cpu()
                reward = torch.FloatTensor([r_batch[i][j]]).cpu()
                done = d_batch[i][j]  # .astype(int)



                learner_append_sample(rep_buffer, policy_net, target_net, state, a_index, reward, next_state, done)
                episode_start_ts += 1
                parse_ts += 1

                # if episode done we reset
                if done:
                    break
            print('{} expert samples added.'.format(episode_start_ts))

        gc.collect()
        print('Batch Parsed finished. {} expert samples added.'.format(parse_ts))


In [6]:
ray.init()

#하이퍼 파라미터
learning_rate = 0.0003
gamma = 0.99
buffer_limit = 50000
L1 = 0.9
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

root_path = os.curdir
model_path = root_path + '/dqn_model/'

2021-06-06 23:42:48,384	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265


In [7]:
policy_net = DQN(19).cuda()
target_net = DQN(19).cuda()
target_net.load_state_dict(policy_net.state_dict())
memory = Memory.remote(30000)
demos = Memory.remote(10000)
optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate, weight_decay=1e-5)

# Copy network params from pretrained Agent
model_path = './dqn_model/per_dqn.pth'
policy_net.load_state_dict(torch.load(model_path, map_location='cuda:0'))
target_net.load_state_dict(policy_net.state_dict())

#parse_demo2.remote("MineRLTreechop-v0", demos, policy_net.cpu(), target_net.cpu(), optimizer, threshold=60, num_epochs=1, batch_size=4, seq_len=60, gamma=0.99, model_name='pre_trained4.pth')


(raylet) /home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   "update your install command.", FutureWarning)


<All keys matched successfully>

In [8]:
# learner network initialzation
batch_size = 256
learner = Learner.remote(policy_net, batch_size)

In [9]:
# actor network, environments initialization
# Generating each own instances

actor1 = Actor.remote(learner, 0, 0.95, 0.05)
actor2 = Actor.remote(learner, 1, 0.5, 0.025)
#actor3 = Actor.remote(learner, 2, 0.1, 0.01)
actor_list = [actor1, actor2]
#actor_list = [actor1, actor2, actor3]

In [10]:
parse = parse_demo.remote("MineRLTreechop-v0", demos, ray.get(learner.get_network.remote()), ray.get(learner.get_target_network.remote()), threshold=40, num_epochs=1, batch_size=4,
              seq_len=400, gamma=0.99)


(pid=93773) Memory is initialized
(pid=93774) Memory is initialized


(pid=93776) /home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=93776)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=93775) /home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=93775)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


(pid=93772) return learner network


(pid=93772) /home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=93772)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=93769) /home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=93769)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=93773) /home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=93773)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=93771) /home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=93771)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=93767) /home/kukjin/anaconda3/e

(pid=93771) data loading sucess
(pid=93771) 0
(pid=93771) 1
(pid=93771) 56.0
(pid=93771) 395 expert samples added.
(pid=93771) 390 expert samples added.
(pid=93771) 389 expert samples added.
(pid=93771) 359 expert samples added.
(pid=93771) Batch Parsed finished. 1533 expert samples added.
(pid=93771) 1533
(pid=93771) 2
(pid=93771) 58.0
(pid=93771) 399 expert samples added.
(pid=93771) 400 expert samples added.
(pid=93771) 354 expert samples added.


In [11]:
explore = [actor.explore.remote(learner, memory) for actor in actor_list]


(pid=93771) 391 expert samples added.
(pid=93771) Batch Parsed finished. 3077 expert samples added.
(pid=93771) 3077
(pid=93771) 3
(pid=93771) 62.0
(pid=93771) 398 expert samples added.
(pid=93771) 383 expert samples added.
(pid=93771) 384 expert samples added.
(pid=93771) 358 expert samples added.
(pid=93771) Batch Parsed finished. 4600 expert samples added.
(pid=93771) 4600
(pid=93771) 4
(pid=93771) 52.0
(pid=93771) 380 expert samples added.
(pid=93771) 390 expert samples added.
(pid=93771) 394 expert samples added.
(pid=93771) 364 expert samples added.
(pid=93771) Batch Parsed finished. 6128 expert samples added.
(pid=93771) 6128
(pid=93771) 5
(pid=93771) 45.0
(pid=93771) 394 expert samples added.
(pid=93771) 374 expert samples added.
(pid=93771) 238 expert samples added.
(pid=93771) 345 expert samples added.
(pid=93771) Batch Parsed finished. 7479 expert samples added.
(pid=93771) 7479
(pid=93771) 6
(pid=93771) 55.0
(pid=93771) 392 expert samples added.
(pid=93771) 372 expert sampl

(pid=93776) MineRL agent is public, connect on port 12340 with Minecraft 1.11
(pid=93775) MineRL agent is public, connect on port 12341 with Minecraft 1.11


In [12]:
update = learner.update_network.remote(memory, demos, batch_size, optimizer)


(pid=93772) started
(pid=93772) 1689
(pid=93772) 10000
(pid=93772) agent batch len : 256 
(pid=93772) demo batch len : 256 


(pid=93772) /home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/ray/workers/default_worker.py:87: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
(pid=93772)   default="WORKER",


(pid=93772) agent batch len : 256 
(pid=93772) demo batch len : 256 
(pid=93772) agent batch len : 256 
(pid=93772) demo batch len : 256 
(pid=93772) agent batch len : 256 
(pid=93772) demo batch len : 256 
(pid=93772) agent batch len : 256 
(pid=93772) demo batch len : 256 
(pid=93772) agent batch len : 256 
(pid=93772) demo batch len : 256 
(pid=93772) agent batch len : 256 
(pid=93772) demo batch len : 256 
(pid=93772) agent batch len : 256 
(pid=93772) demo batch len : 256 
(pid=93772) agent batch len : 256 
(pid=93772) demo batch len : 256 
(pid=93772) agent batch len : 256 
(pid=93772) demo batch len : 256 
(pid=93772) Count : 10 leaner_network updated
(pid=93772) agent batch len : 256 
(pid=93772) demo batch len : 256 
(pid=93772) agent batch len : 256 
(pid=93772) demo batch len : 256 
(pid=93772) agent batch len : 256 
(pid=93772) demo batch len : 256 
(pid=93772) agent batch len : 256 
(pid=93772) demo batch len : 256 
(pid=93772) agent batch len : 256 
(pid=93772) demo batch

(pid=93774) /home/kukjin/Study/RL/softwardcapstonedesign/ApexDQFD/utils.py:101: RuntimeWarning: divide by zero encountered in power
(pid=93774)   is_weight = np.power(self.tree.n_entries * sampling_probabilities, -self.beta)
(pid=93774) /home/kukjin/Study/RL/softwardcapstonedesign/ApexDQFD/utils.py:102: RuntimeWarning: invalid value encountered in true_divide
(pid=93774)   is_weight /= (is_weight.max() + 1e-5)


(pid=93772) agent batch len : 256 
(pid=93772) return learner network
(pid=93772) return learner network
(pid=93776) actor network is updated 
(pid=93775) actor network is updated 


2021-06-06 23:45:34,290	ERROR worker.py:1056 -- Possible unhandled error from worker: ray::Learner.update_network() (pid=93772, ip=192.168.0.22)
  File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
  File "/home/kukjin/anaconda3/envs/minerl/lib/python3.7/site-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
    return method(__ray_actor, *args, **kwargs)
  File "<ipython-input-4-4b5e34112ebc>", line 56, in update_network
TypeError: cannot unpack non-iterable int object
(pid=93776) MineRL agent is public, connect on port 12340 with Minecraft 1.11
